# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.sort_values('Metropolitan area')
    nhl_df.drop([0,9,18,26,35,44,53,61,69,78,87,95,103,112,121,129,137,146,155,163], axis = 0, inplace = True)
    nhl_df = nhl_df.astype({'GP': int, 'W': int, 'L': int, 'OL': int, 'PTS': int, 'PTS%': float, 'GF': int, 'GA': int, 'SRS': float, 'SOS': float, 'RPt%': float, 'ROW': int, 'year': int})
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    cities = cities.sort_values('Metropolitan area')
    cities['NHL'] = cities['NHL'].replace('—', np.NaN)
    cities = cities.dropna()
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    lst_1 = []
    for i in cities['NHL']:
        i = re.findall('^\w*', i)
        lst_1.append(i[0])
    cities['NHL'] = lst_1
    cities.replace('', np.NaN, inplace = True)
    cities = cities.dropna()
    lst = []    
    for i in nhl_df['team']:
        i = re.findall('[^*]+', i)
        lst.append(i[0])
    nhl_df['team'] = lst
    nhl_df['team_ville'] = nhl_df['team'].replace({'Tampa Bay Lightning':'Tampa Bay Area',
     'Boston Bruins':'Boston',
     'Toronto Maple Leafs':'Toronto',
     'Florida Panthers':'Miami–Fort Lauderdale',
     'Detroit Red Wings':'Detroit',
     'Montreal Canadiens':'Montreal',
     'Ottawa Senators':'Ottawa',
     'Buffalo Sabres':'Buffalo',
     'Washington Capitals':'Washington, D.C.',
     'Pittsburgh Penguins':'Pittsburgh',
     'Philadelphia Flyers':'Philadelphia',
     'Columbus Blue Jackets':'Columbus',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Nashville Predators':'Nashville',
     'Winnipeg Jets':'Winnipeg',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'St. Louis Blues':'St. Louis',
     'Dallas Stars':'Dallas–Fort Worth',
     'Chicago Blackhawks':'Chicago',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'San Jose Sharks':'San Francisco Bay Area',
     'Los Angeles Kings':'Los Angeles',
     'Calgary Flames':'Calgary',
     'Edmonton Oilers':'Edmonton',
     'Vancouver Canucks':'Vancouver',
     'Arizona Coyotes':'Phoenix'})
    cities.sort_values(by = 'Metropolitan area', inplace = True)
    nhl_df1 = nhl_df.sort_values(by = 'team_ville')
    y_wlr = nhl_df[nhl_df['year'] == 2018]
    y_wlr = y_wlr[['W','L','team_ville']]
    y_wlr = y_wlr.groupby('team_ville').sum()
    games = y_wlr['W'] + y_wlr['L']
    y_wlr['WLR'] = y_wlr["W"]/games
    
    df = pd.merge(y_wlr, cities, how = 'inner', left_on = 'team_ville', right_on = 'Metropolitan area')
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df['WLR']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012308996455744249

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    # YOUR CODE HERE
    import pandas as pd
    import numpy as np
    import re
    nba_df=pd.read_csv("assets/nba.csv")
    cities_2=pd.read_html("assets/wikipedia_data.html")[1]
    cities_2=cities_2.iloc[:-1,[0,3,5,6,7,8]]
    cities_2 = cities_2.sort_values('Metropolitan area')
    cities_2 = cities_2[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    lst_1 = []
    for i in cities_2['NBA']:
        i = re.findall('^\w*', i)
        lst_1.append(i[0])
    cities_2['NBA'] = lst_1
    cities_2['NBA'] = cities_2['NBA'].replace('', np.NaN)
    cities_2 = cities_2.dropna()
    nba_df.drop([90,96,102,108,114,120,126,132,138,144,150,156], axis = 0, inplace = True)
    cities_2['Population (2016 est.)[8]'] = pd.to_numeric(cities_2['Population (2016 est.)[8]'])
    lst = []
    for i in nba_df['team']:
        i = re.findall('[^*]+', i)
        lst.append(i[0])
    nba_df['team'] = lst
    lst_1 = []
    for i in nba_df['team']:
        i = i.split('(')
        lst_1.append(i[0])
    nba_df['team'] = lst_1
    lst_2 = []
    for i in nba_df['team']:
        i = i.rstrip()
        lst_2.append(i)
    
    nba_df['team'] = lst_2
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team_ville'] = nba_df['team'].map({'Toronto Raptors':'Toronto',
     'Boston Celtics':'Boston',
     'Philadelphia 76ers':'Philadelphia',
     'Cleveland Cavaliers':'Cleveland',
     'Indiana Pacers':'Indianapolis',
     'Miami Heat':'Miami–Fort Lauderdale',
     'Milwaukee Bucks':'Milwaukee',
     'Washington Wizards':'Washington, D.C.',
     'Detroit Pistons':'Detroit',
     'Charlotte Hornets':'Charlotte',
     'New York Knicks':'New York City',
     'Brooklyn Nets':'New York City',
     'Chicago Bulls':'Chicago',
     'Orlando Magic':'Orlando',
     'Atlanta Hawks':'Atlanta',
     'Houston Rockets':'Houston',
     'Golden State Warriors':'San Francisco Bay Area',
     'Portland Trail Blazers':'Portland',
     'Oklahoma City Thunder':'Oklahoma City',
     'Utah Jazz':'Salt Lake City',
     'New Orleans Pelicans':'New Orleans',
     'San Antonio Spurs':'San Antonio',
     'Minnesota Timberwolves':'Minneapolis–Saint Paul',
     'Denver Nuggets':'Denver',
     'Los Angeles Clippers':'Los Angeles',
     'Los Angeles Lakers':'Los Angeles',
     'Sacramento Kings':'Sacramento',
     'Dallas Mavericks':'Dallas–Fort Worth',
     'Memphis Grizzlies':'Memphis',
     'Phoenix Suns':'Phoenix'})
    nba_df['W'] = pd.to_numeric(nba_df['W'])
    nba_df['L'] = pd.to_numeric(nba_df['L'])
    nba_df['W/L%'] = pd.to_numeric(nba_df['W/L%'])
    nba_df = nba_df.groupby('team_ville').mean()
    cities_2['Population (2016 est.[8])'] = pd.to_numeric(cities_2['Population (2016 est.)[8]'])
    df1 = pd.merge(cities_2, nba_df, how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    
    population_by_region = cities_2['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df1['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:

def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df['W'] =pd.to_numeric(mlb_df['W'])
    mlb_df['L'] = pd.to_numeric(mlb_df['L'])
    mlb_df['W-L%'] = pd.to_numeric(mlb_df['W-L%'])
    mlb_df['team_ville'] = mlb_df['team'].map({'Boston Red Sox':'Boston',
     'New York Yankees':'New York City',
     'Tampa Bay Rays':'Tampa Bay Area',
     'Toronto Blue Jays':'Toronto',
     'Baltimore Orioles':'Baltimore',
     'Cleveland Indians':'Cleveland',
     'Minnesota Twins':'Minneapolis–Saint Paul',
     'Detroit Tigers':'Detroit',
     'Chicago White Sox':'Chicago',
     'Kansas City Royals':'Kansas City',
     'Houston Astros':'Houston',
     'Oakland Athletics':'San Francisco Bay Area',
     'Seattle Mariners':'Seattle',
     'Los Angeles Angels':'Los Angeles',
     'Texas Rangers':'Dallas–Fort Worth',
     'Atlanta Braves':'Atlanta',
     'Washington Nationals':'Washington, D.C.',
     'Philadelphia Phillies':'Philadelphia',
     'New York Mets':'New York City',
     'Miami Marlins':'Miami–Fort Lauderdale',
     'Milwaukee Brewers':'Milwaukee',
     'Chicago Cubs':'Chicago',
     'St. Louis Cardinals':'St. Louis',
     'Pittsburgh Pirates':'Pittsburgh',
     'Cincinnati Reds':'Cincinnati',
     'Los Angeles Dodgers':'Los Angeles',
     'Colorado Rockies':'Denver',
     'Arizona Diamondbacks':'Phoenix',
     'San Francisco Giants':'San Francisco Bay Area',
     'San Diego Padres':'San Diego'})
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    cities = cities.sort_values('Metropolitan area')
    mlb_df = mlb_df.sort_values('team_ville')
    l4 =[]
    for i in cities['MLB']:
        i = re.findall('^\w*',i)
        l4.append(i[0])
    cities['MLB'] = l4
    cities['MLB'] = cities['MLB'].replace('', np.NaN)
    cities= cities.dropna()
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    df2 = pd.merge(cities, mlb_df, how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    df2 = df2.groupby('Metropolitan area').mean()
    # YOUR CODE HERE    
    population_by_region = cities['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df2['W-L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:

def nfl_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities_4=pd.read_html("assets/wikipedia_data.html")[1]
    cities_4=cities_4.iloc[:-1,[0,3,5,6,7,8]]
    cols = np.arange(0,200,5)
    nfl_df = nfl_df.drop(cols,axis=0)
    l5 = []
    for i in nfl_df['team']:
        i = re.findall('[^*|+]+', i)
        l5.append(i[0])
    nfl_df['team'] = l5
    nfl_df['team_ville'] = nfl_df['team'].map({'New England Patriots':'Boston',
     'Miami Dolphins':'Miami–Fort Lauderdale',
     'Buffalo Bills':'Buffalo',
     'New York Jets':'New York City',
     'Baltimore Ravens':'Baltimore',
     'Pittsburgh Steelers':'Pittsburgh',
     'Cleveland Browns':'Cleveland',
     'Cincinnati Bengals':'Cincinnati',
     'Houston Texans':'Houston',
     'Indianapolis Colts':'Indianapolis',
     'Tennessee Titans':'Nashville',
     'Jacksonville Jaguars':'Jacksonville',
     'Kansas City Chiefs':'Kansas City',
     'Los Angeles Chargers':'Los Angeles',
     'Denver Broncos':'Denver',
     'Oakland Raiders':'San Francisco Bay Area',
     'Dallas Cowboys':'Dallas–Fort Worth',
     'Philadelphia Eagles':'Philadelphia',
     'Washington Redskins':'Washington, D.C.',
     'New York Giants':'New York City',
     'Chicago Bears':'Chicago',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Green Bay Packers':'Green Bay',
     'Detroit Lions':'Detroit',
     'New Orleans Saints':'New Orleans',
     'Carolina Panthers':'Charlotte',
     'Atlanta Falcons':'Atlanta',
     'Tampa Bay Buccaneers':'Tampa Bay Area',
     'Los Angeles Rams':'Los Angeles',
     'Seattle Seahawks':'Seattle',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'})
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[['team_ville', 'W', 'L', 'W-L%', 'year', 'team']]
    cities_4 = cities_4[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    cities_4['Population (2016 est.)[8]'] = pd.to_numeric(cities_4['Population (2016 est.)[8]'])
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    nfl_df['W-L%'] = pd.to_numeric(nfl_df['W-L%'])
    l6 = []
    for i in cities_4['NFL']:
        i = re.findall('^\w*', i)
        l6.append(i[0])
    cities_4['NFL'] = l6
    cities_4 = cities_4.replace('', np.NaN)
    cities_4 = cities_4.dropna()
    df3 = pd.merge(cities_4,nfl_df,how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    df3 = df3.groupby('Metropolitan area').mean()
    # YOUR CODE HERE    
    population_by_region = cities_4['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = df3['W-L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nfl_correlation()

0.0014031100865728862

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import ttest_rel
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities = cities.sort_values('Metropolitan area')
    nhl_df.drop([0,9,18,26,35,44,53,61,69,78,87,95,103,112,121,129,137,146,155,163], axis = 0, inplace = True)
    nhl_df = nhl_df.astype({'GP': int, 'W': int, 'L': int, 'OL': int, 'PTS': int, 'PTS%': float, 'GF': int, 'GA': int, 'SRS': float, 'SOS': float, 'RPt%': float, 'ROW': int, 'year': int})
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    cities = cities.sort_values('Metropolitan area')
    cities['NHL'] = cities['NHL'].replace('—', np.NaN)
    cities = cities.dropna()
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    lst_1 = []
    for i in cities['NHL']:
        i = re.findall('^\w*', i)
        lst_1.append(i[0])
    cities['NHL'] = lst_1
    cities.replace('', np.NaN, inplace = True)
    cities = cities.dropna()
    lst = []    
    for i in nhl_df['team']:
        i = re.findall('[^*]+', i)
        lst.append(i[0])
    nhl_df['team'] = lst
    nhl_df['team_ville'] = nhl_df['team'].replace({'Tampa Bay Lightning':'Tampa Bay Area',
     'Boston Bruins':'Boston',
     'Toronto Maple Leafs':'Toronto',
     'Florida Panthers':'Miami–Fort Lauderdale',
     'Detroit Red Wings':'Detroit',
     'Montreal Canadiens':'Montreal',
     'Ottawa Senators':'Ottawa',
     'Buffalo Sabres':'Buffalo',
     'Washington Capitals':'Washington, D.C.',
     'Pittsburgh Penguins':'Pittsburgh',
     'Philadelphia Flyers':'Philadelphia',
     'Columbus Blue Jackets':'Columbus',
     'New Jersey Devils':'New York City',
     'Carolina Hurricanes':'Raleigh',
     'New York Islanders':'New York City',
     'New York Rangers':'New York City',
     'Nashville Predators':'Nashville',
     'Winnipeg Jets':'Winnipeg',
     'Minnesota Wild':'Minneapolis–Saint Paul',
     'Colorado Avalanche':'Denver',
     'St. Louis Blues':'St. Louis',
     'Dallas Stars':'Dallas–Fort Worth',
     'Chicago Blackhawks':'Chicago',
     'Vegas Golden Knights':'Las Vegas',
     'Anaheim Ducks':'Los Angeles',
     'San Jose Sharks':'San Francisco Bay Area',
     'Los Angeles Kings':'Los Angeles',
     'Calgary Flames':'Calgary',
     'Edmonton Oilers':'Edmonton',
     'Vancouver Canucks':'Vancouver',
     'Arizona Coyotes':'Phoenix'})
    cities.sort_values(by = 'Metropolitan area', inplace = True)
    nhl_df1 = nhl_df.sort_values(by = 'team_ville')
    y_wlr = nhl_df[nhl_df['year'] == 2018]
    y_wlr = y_wlr[['W','L','team_ville']]
    y_wlr = y_wlr.groupby('team_ville').sum()
    games = y_wlr['W'] + y_wlr['L']
    y_wlr['WLR'] = y_wlr["W"]/games
    
    df = pd.merge(y_wlr, cities, how = 'inner', left_on = 'team_ville', right_on = 'Metropolitan area')
    return df     
   
def nba_correlation():
    # YOUR CODE HERE
    import pandas as pd
    import numpy as np
    import re
    nba_df=pd.read_csv("assets/nba.csv")
    cities_2=pd.read_html("assets/wikipedia_data.html")[1]
    cities_2=cities_2.iloc[:-1,[0,3,5,6,7,8]]
    cities_2 = cities_2.sort_values('Metropolitan area')
    cities_2 = cities_2[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    lst_1 = []
    for i in cities_2['NBA']:
        i = re.findall('^\w*', i)
        lst_1.append(i[0])
    cities_2['NBA'] = lst_1
    cities_2['NBA'] = cities_2['NBA'].replace('', np.NaN)
    cities_2 = cities_2.dropna()
    nba_df.drop([90,96,102,108,114,120,126,132,138,144,150,156], axis = 0, inplace = True)
    cities_2['Population (2016 est.)[8]'] = pd.to_numeric(cities_2['Population (2016 est.)[8]'])
    lst = []
    for i in nba_df['team']:
        i = re.findall('[^*]+', i)
        lst.append(i[0])
    nba_df['team'] = lst
    lst_1 = []
    for i in nba_df['team']:
        i = i.split('(')
        lst_1.append(i[0])
    nba_df['team'] = lst_1
    lst_2 = []
    for i in nba_df['team']:
        i = i.rstrip()
        lst_2.append(i)
    
    nba_df['team'] = lst_2
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team_ville'] = nba_df['team'].map({'Toronto Raptors':'Toronto',
     'Boston Celtics':'Boston',
     'Philadelphia 76ers':'Philadelphia',
     'Cleveland Cavaliers':'Cleveland',
     'Indiana Pacers':'Indianapolis',
     'Miami Heat':'Miami–Fort Lauderdale',
     'Milwaukee Bucks':'Milwaukee',
     'Washington Wizards':'Washington, D.C.',
     'Detroit Pistons':'Detroit',
     'Charlotte Hornets':'Charlotte',
     'New York Knicks':'New York City',
     'Brooklyn Nets':'New York City',
     'Chicago Bulls':'Chicago',
     'Orlando Magic':'Orlando',
     'Atlanta Hawks':'Atlanta',
     'Houston Rockets':'Houston',
     'Golden State Warriors':'San Francisco Bay Area',
     'Portland Trail Blazers':'Portland',
     'Oklahoma City Thunder':'Oklahoma City',
     'Utah Jazz':'Salt Lake City',
     'New Orleans Pelicans':'New Orleans',
     'San Antonio Spurs':'San Antonio',
     'Minnesota Timberwolves':'Minneapolis–Saint Paul',
     'Denver Nuggets':'Denver',
     'Los Angeles Clippers':'Los Angeles',
     'Los Angeles Lakers':'Los Angeles',
     'Sacramento Kings':'Sacramento',
     'Dallas Mavericks':'Dallas–Fort Worth',
     'Memphis Grizzlies':'Memphis',
     'Phoenix Suns':'Phoenix'})
    nba_df['W'] = pd.to_numeric(nba_df['W'])
    nba_df['L'] = pd.to_numeric(nba_df['L'])
    nba_df['W/L%'] = pd.to_numeric(nba_df['W/L%'])
    nba_df = nba_df.groupby('team_ville').mean()
    cities_2['Population (2016 est.[8])'] = pd.to_numeric(cities_2['Population (2016 est.)[8]'])
    df1 = pd.merge(cities_2, nba_df, how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    return df1

def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df['W'] =pd.to_numeric(mlb_df['W'])
    mlb_df['L'] = pd.to_numeric(mlb_df['L'])
    mlb_df['W-L%'] = pd.to_numeric(mlb_df['W-L%'])
    mlb_df['team_ville'] = mlb_df['team'].map({'Boston Red Sox':'Boston',
     'New York Yankees':'New York City',
     'Tampa Bay Rays':'Tampa Bay Area',
     'Toronto Blue Jays':'Toronto',
     'Baltimore Orioles':'Baltimore',
     'Cleveland Indians':'Cleveland',
     'Minnesota Twins':'Minneapolis–Saint Paul',
     'Detroit Tigers':'Detroit',
     'Chicago White Sox':'Chicago',
     'Kansas City Royals':'Kansas City',
     'Houston Astros':'Houston',
     'Oakland Athletics':'San Francisco Bay Area',
     'Seattle Mariners':'Seattle',
     'Los Angeles Angels':'Los Angeles',
     'Texas Rangers':'Dallas–Fort Worth',
     'Atlanta Braves':'Atlanta',
     'Washington Nationals':'Washington, D.C.',
     'Philadelphia Phillies':'Philadelphia',
     'New York Mets':'New York City',
     'Miami Marlins':'Miami–Fort Lauderdale',
     'Milwaukee Brewers':'Milwaukee',
     'Chicago Cubs':'Chicago',
     'St. Louis Cardinals':'St. Louis',
     'Pittsburgh Pirates':'Pittsburgh',
     'Cincinnati Reds':'Cincinnati',
     'Los Angeles Dodgers':'Los Angeles',
     'Colorado Rockies':'Denver',
     'Arizona Diamondbacks':'Phoenix',
     'San Francisco Giants':'San Francisco Bay Area',
     'San Diego Padres':'San Diego'})
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    cities = cities.sort_values('Metropolitan area')
    mlb_df = mlb_df.sort_values('team_ville')
    l4 =[]
    for i in cities['MLB']:
        i = re.findall('^\w*',i)
        l4.append(i[0])
    cities['MLB'] = l4
    cities['MLB'] = cities['MLB'].replace('', np.NaN)
    cities= cities.dropna()
    cities['Population (2016 est.)[8]'] = pd.to_numeric(cities['Population (2016 est.)[8]'])
    df2 = pd.merge(cities, mlb_df, how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    df2 = df2.groupby('Metropolitan area').mean()
    # YOUR CODE HERE    
    return df2
    
def nfl_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nfl_df=pd.read_csv("assets/nfl.csv")
    cities_4=pd.read_html("assets/wikipedia_data.html")[1]
    cities_4=cities_4.iloc[:-1,[0,3,5,6,7,8]]
    cols = np.arange(0,200,5)
    nfl_df = nfl_df.drop(cols,axis=0)
    l5 = []
    for i in nfl_df['team']:
        i = re.findall('[^*|+]+', i)
        l5.append(i[0])
    nfl_df['team'] = l5
    nfl_df['team_ville'] = nfl_df['team'].map({'New England Patriots':'Boston',
     'Miami Dolphins':'Miami–Fort Lauderdale',
     'Buffalo Bills':'Buffalo',
     'New York Jets':'New York City',
     'Baltimore Ravens':'Baltimore',
     'Pittsburgh Steelers':'Pittsburgh',
     'Cleveland Browns':'Cleveland',
     'Cincinnati Bengals':'Cincinnati',
     'Houston Texans':'Houston',
     'Indianapolis Colts':'Indianapolis',
     'Tennessee Titans':'Nashville',
     'Jacksonville Jaguars':'Jacksonville',
     'Kansas City Chiefs':'Kansas City',
     'Los Angeles Chargers':'Los Angeles',
     'Denver Broncos':'Denver',
     'Oakland Raiders':'San Francisco Bay Area',
     'Dallas Cowboys':'Dallas–Fort Worth',
     'Philadelphia Eagles':'Philadelphia',
     'Washington Redskins':'Washington, D.C.',
     'New York Giants':'New York City',
     'Chicago Bears':'Chicago',
     'Minnesota Vikings':'Minneapolis–Saint Paul',
     'Green Bay Packers':'Green Bay',
     'Detroit Lions':'Detroit',
     'New Orleans Saints':'New Orleans',
     'Carolina Panthers':'Charlotte',
     'Atlanta Falcons':'Atlanta',
     'Tampa Bay Buccaneers':'Tampa Bay Area',
     'Los Angeles Rams':'Los Angeles',
     'Seattle Seahawks':'Seattle',
     'San Francisco 49ers':'San Francisco Bay Area',
     'Arizona Cardinals':'Phoenix'})
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df = nfl_df[['team_ville', 'W', 'L', 'W-L%', 'year', 'team']]
    cities_4 = cities_4[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    cities_4['Population (2016 est.)[8]'] = pd.to_numeric(cities_4['Population (2016 est.)[8]'])
    nfl_df['W'] = pd.to_numeric(nfl_df['W'])
    nfl_df['L'] = pd.to_numeric(nfl_df['L'])
    nfl_df['W-L%'] = pd.to_numeric(nfl_df['W-L%'])
    l6 = []
    for i in cities_4['NFL']:
        i = re.findall('^\w*', i)
        l6.append(i[0])
    cities_4['NFL'] = l6
    cities_4 = cities_4.replace('', np.NaN)
    cities_4 = cities_4.dropna()
    df3 = pd.merge(cities_4,nfl_df,how = 'inner', left_on = 'Metropolitan area', right_on = 'team_ville')
    df3 = df3.groupby('Metropolitan area').mean()
    # YOUR CODE HERE    
    return df3
    raise NotImplementedError()


def sports_team_performance():
    # YOUR CODE HERE
    nba = nba_correlation()
    mlb = mlb_correlation()
    nhl = nhl_correlation()
    nfl = nfl_correlation()
    
    nba_nhl = pd.merge(nba, nhl, on='Metropolitan area')
    mlb_nfl = pd.merge(mlb,nfl, on = 'Metropolitan area')
    nhl_nfl = pd.merge(nhl, nfl, on = 'Metropolitan area')
    nba_nfl = pd.merge(nba,nfl, on = 'Metropolitan area')
    nba_mlb = pd.merge(nba, mlb, on= 'Metropolitan area')
    mlb_nhl = pd.merge(mlb,nhl, on = 'Metropolitan area')
    
    pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L%'], nba_nhl['WLR'])
    pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W-L%_x'], mlb_nfl['W-L%_y'])
    pval_nhl_nfl = stats.ttest_rel(nhl_nfl['WLR'], nhl_nfl['W-L%'])
    pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L%'], nba_nfl['W-L%'])
    pval_nba_mlb = stats.ttest_rel(nba_mlb['W/L%'], nba_mlb['W-L%'])
    pval_mlb_nhl = stats.ttest_rel(mlb_nhl['WLR'], mlb_nhl['W-L%'])
       
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    pv = {'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
       'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_nba_mlb},
       'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
       'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_nba_mlb, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
      }

    p_values = pd.DataFrame(pv)
    
    assert abs(p_values.loc["NBA", "NHL"][1] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"][1] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    raise NotImplementedError()

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,"(-0.07950419757855262, 0.9375088282360183)","(2.375006146319349, 0.030391975468575805)","(-0.25204162096749955, 0.8034592132146956)"
NBA,"(-0.07950419757855262, 0.9375088282360183)",NaN,"(-2.5905361064540027, 0.02240459747249962)","(0.06419249931541132, 0.9495657419262777)"
NHL,"(2.375006146319349, 0.030391975468575805)","(-2.5905361064540027, 0.02240459747249962)",NaN,"(4.1827821242869625, 0.0007033745889305857)"
MLB,"(-0.25204162096749955, 0.8034592132146956)","(0.06419249931541132, 0.9495657419262777)","(4.1827821242869625, 0.0007033745889305857)",NaN
